In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import lightgbm as lgbm
import gc
import xgboost as xgb
import pickle as pickle


from catboost import CatBoostRegressor
import dask.dataframe as dd
from sklearn.model_selection import KFold

In [4]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [5]:
y_cols = ['shop_id', 'item_id', 'date_block_num']

In [6]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'] != 2013]


In [7]:
train_item_ids = sales_train['item_id'].unique()
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [8]:
combinations = []
for shop in all_shop_ids:
    #get all article ids ever associated to this shop
    train_ids = sales_train[sales_train['shop_id'] == shop]['item_id'].unique()
    test_ids = sales_train[sales_train['shop_id'] == shop]['item_id'].unique()
    all_shop = np.unique(np.append(train_ids, test_ids))
    all_shop_combo = [[shop, item, block] for item in all_shop for block in train_blocks]
    for combo in all_shop_combo:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])
del combinations

In [9]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"y"})

In [161]:
training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)

In [173]:
training = training.apply(pd.to_numeric,downcast='unsigned')

training['y'] = training['y'].clip(0,20)
training['y'] = training['y'].astype('int8')

In [174]:
training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

In [175]:
def get_mean_encoding(df, group_cols, target):
    cumsum = df.groupby(group_cols)[target].cumsum() - df[target]
    cumcnt = df.groupby(group_cols).cumcount()
    return cumsum/cumcnt
    

In [176]:
training['item_me'] = pd.to_numeric(get_mean_encoding(training, ['item_id'], 'y'), downcast='float')
training['shop_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id'], 'y'), downcast='float')
training['category_me'] = pd.to_numeric(get_mean_encoding(training, ['item_category_id'], 'y'), downcast='float')
training['shop_category_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id', 'item_category_id'], 'y'), downcast='float')
training['shop_item_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id', 'item_id'], 'y'), downcast='float')

training.fillna(0,inplace=True)

In [177]:
training.dtypes

item_id                        int64
shop_id                        uint8
date_block_num                 uint8
y                               int8
item_block_units             float64
level_1_x                     uint32
item_block_units_rolling3    float64
shop_block_units             float64
level_1_y                     uint32
shop_block_units_rolling3    float64
item_category_id               int64
item_me                      float32
shop_me                      float32
category_me                  float32
shop_category_me             float32
shop_item_me                 float32
dtype: object

In [42]:
training.sample(10)

,item_id,shop_id,date_block_num,y,item_category_id,item_me,shop_me,category_me,val_ignore,shop_category_me,shop_item_me
5434358,20056,27,18,0,37,0.102941,0.460179,0.159993,False,0.241283,0.000000
4799148,17808,58,14,0,40,0.077128,0.393490,0.228244,True,0.380209,0.500000
4444493,16502,6,21,0,37,0.056604,0.311204,0.160704,True,0.185232,0.111111
4176272,15689,4,24,0,37,0.058824,0.248767,0.159517,False,0.076485,0.083333
4350554,16169,50,22,4,65,1.895075,0.286752,0.536985,False,0.296774,1.400000
763289,3340,26,33,1,30,3.403471,0.348834,0.883223,True,0.768427,3.619048
2337869,9181,46,29,0,30,0.074928,0.444447,0.830466,True,1.016901,0.117647
3111310,12092,30,16,1,37,0.076923,0.233815,0.168609,True,0.090826,0.000000
1788611,7053,2,23,0,23,0.181818,0.392188,0.474803,True,0.442603,0.181818
4184866,15721,3,16,0,37,0.115385,0.236361,0.159772,True,0.084404,0.500000


In [31]:
transactions = sales_train


transactions = transactions.set_index('item_id').join(items.set_index('item_id'))
transactions.reset_index(inplace=True)
transactions = transactions[transactions['year'] != 2013]



In [21]:
number_of_items = transactions['item_id'].nunique()
print("number_of_items:", number_of_items)
number_of_categories = transactions['item_category_id'].nunique()
print("number_of_categories:", number_of_categories)
number_of_shops = transactions['shop_id'].nunique()
print("number_of_shops:", number_of_shops)
number_of_days = 365 + 365 - 30 - 31
print("number_of_days:", number_of_days)
number_of_blocks = transactions['date_block_num'].nunique()
print("number_of_blocks:", number_of_blocks)
total_sales = transactions['item_cnt_day'].sum()
print("total_sales:", total_sales)
total_turnover = transactions['turnover'].sum()
print("total_turnover:", total_turnover)
average_price = transactions['item_price'].mean()
print("average_price:", average_price)

number_of_items: 17054
number_of_categories: 79
number_of_shops: 55
number_of_days: 669
number_of_blocks: 22
total_sales: 2085473.0
total_turnover: 2181401610.589987
average_price: 1015.5023073772021


In [32]:
#ITEM

gc.collect()
transactions_items_blocks = transactions.copy()

transactions_items_blocks['item_block_units'] = pd.to_numeric(transactions_items_blocks\
                .groupby(['item_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')    

transactions_items_blocks.drop_duplicates(['item_id','date_block_num'],inplace=True)

In [162]:
items_blocks = transactions_items_blocks[['item_id', 'date_block_num', 'item_block_units']]
training = training.merge(items_blocks, on=['item_id', 'date_block_num'], how='left').fillna(0)

In [163]:
item_block_units_rolling3 = training\
        .sort_values(['item_id','date_block_num'])\
        .drop_duplicates(['item_id','date_block_num'])\
        .groupby(['item_id'])\
        ['item_block_units'].rolling(3,min_periods=2).mean().reset_index()\
        .rename(columns={'item_block_units':'item_block_units_rolling3'})

item_block_units_rolling3['date_block_num'] = item_block_units_rolling3['level_1'].map(training['date_block_num'])



training = training.merge(item_block_units_rolling3, on=['item_id', 'date_block_num'], how='left').fillna(0)

In [165]:
#SHOP

gc.collect()
transactions_shops_blocks = transactions.copy()

transactions_shops_blocks['shop_block_units'] = pd.to_numeric(transactions_shops_blocks\
                .groupby(['shop_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')    

transactions_shops_blocks.drop_duplicates(['shop_id','date_block_num'],inplace=True)

In [166]:
shops_blocks = transactions_shops_blocks[['shop_id', 'date_block_num', 'shop_block_units']]
training = training.merge(shops_blocks, on=['shop_id', 'date_block_num'], how='left').fillna(0)

In [167]:
shop_block_units_rolling3 = training\
        .sort_values(['shop_id','date_block_num'])\
        .drop_duplicates(['shop_id','date_block_num'])\
        .groupby(['shop_id'])\
        ['shop_block_units'].rolling(3,min_periods=2).mean().reset_index()\
        .rename(columns={'shop_block_units':'shop_block_units_rolling3'})

shop_block_units_rolling3['date_block_num'] = shop_block_units_rolling3['level_1'].map(training['date_block_num'])



training = training.merge(shop_block_units_rolling3, on=['shop_id', 'date_block_num'], how='left').fillna(0)

In [212]:
training['rolling_composite'] =  training['shop_block_units_rolling3'].clip(1, None) *\
            training['item_block_units_rolling3'].clip(1, None) 
training['me_composite'] =  training['item_me'].clip(1, None)  * training['shop_me'].clip(1, None) 

In [164]:
training[(training['item_id'].isin([30,31])) & (training['shop_id'] == 30)]\
        .sort_values(['item_id','date_block_num'])[['item_id','shop_id',\
                                                    'date_block_num','item_block_units', 'item_block_units_rolling3']]

,item_id,shop_id,date_block_num,item_block_units,item_block_units_rolling3
2775850,30,30,12,58.0,0.000000
2775851,30,30,13,24.0,41.000000
2775852,30,30,14,31.0,37.666667
2775853,30,30,15,21.0,25.333333
2775854,30,30,16,16.0,22.666667
2775855,30,30,17,13.0,16.666667
2775856,30,30,18,13.0,14.000000
2775857,30,30,19,12.0,12.666667
2775858,30,30,20,11.0,12.000000
2775859,30,30,21,13.0,12.000000


In [172]:
training[(training['item_id'].isin([30])) & (training['shop_id'].isin([30,31]))]\
        .sort_values(['shop_id','date_block_num'])[['item_id','shop_id',\
                                                    'date_block_num','shop_block_units', 'shop_block_units_rolling3']]

,item_id,shop_id,date_block_num,shop_block_units,shop_block_units_rolling3
2775850,30,30,12,2383.0,0.000000
2775851,30,30,13,2260.0,2321.500000
2775852,30,30,14,2361.0,2334.666667
2775853,30,30,15,1995.0,2205.333333
2775854,30,30,16,1960.0,2105.333333
2775855,30,30,17,1775.0,1910.000000
2775856,30,30,18,1620.0,1785.000000
2775857,30,30,19,1688.0,1694.333333
2775858,30,30,20,1976.0,1761.333333
2775859,30,30,21,1968.0,1877.333333


In [38]:
training.sample(10)

,item_id,shop_id,date_block_num,y,item_category_id,item_me,shop_me,category_me,shop_category_me,shop_item_me,item_block_units
1145279,4628,5,15,1,55,0.144928,0.324163,0.228592,0.166368,0.333333,118.0
3898901,14865,45,29,0,40,0.139579,0.229708,0.233409,0.134075,0.058824,7.0
2745746,10772,51,26,0,55,0.149425,0.216210,0.224970,0.144154,0.071429,10.0
197539,1391,49,13,0,24,0.229508,0.148330,0.229508,0.000000,0.000000,0.0
1574998,6270,35,30,0,55,0.302817,0.457775,0.249876,0.276829,0.555556,7.0
2426490,9728,53,12,0,40,0.060606,0.418135,0.230754,0.174242,0.000000,0.0
2589078,10309,44,20,0,41,0.078947,0.220292,0.137742,0.094937,0.125000,0.0
5405241,19894,5,29,0,37,0.081967,0.239544,0.159903,0.104901,0.176471,6.0
3929184,15023,6,18,1,30,0.500000,0.317615,0.826002,1.001495,0.666667,44.0
4216621,15805,47,25,0,40,0.266430,0.379818,0.232849,0.155026,0.000000,0.0


In [35]:
len(training[training['item_block_units'] > 0])

3456135

In [37]:
len(transactions_items_blocks)

135451

In [178]:
gc.collect()
val = training[training['date_block_num'] == 33]
print("val length", len(val))

unique_pairs_val = list(set(list(zip(val.shop_id, val.item_id))))
print("number of unique shop/item pairs in val", len(unique_pairs_val))
unique_pairs_val_ignore = unique_pairs_val[0:int(len(unique_pairs_val)/2)]


def tuple2key(t):
    return "%d_%d" % (t[0], t[1])

val_pairs_ignore_dict = {}
for t in unique_pairs_val_ignore:
    val_pairs_ignore_dict[tuple2key(t)] = 1
 
    
training['val_ignore'] = (training['shop_id'].astype(str) + '_' +  training['item_id'].astype(str))\
                                    .apply(lambda x: x in val_pairs_ignore_dict)

val length 270742
number of unique shop/item pairs in val 270742


In [101]:
len(training[training['val_ignore'] == True])

2978162

In [213]:
gc.collect()


x_train = training[(training['date_block_num'] < 33) & (training['val_ignore'] == False)]
y_train = x_train['y']


pos_train_len = len(y_train[y_train != 0])
print("pos_train_len", pos_train_len)

zeros_keep_indices_train = y_train[y_train == 0].sample(int(pos_train_len/6)).index
print("zeros_keep_indices_train", len(zeros_keep_indices_train))
non_zeros_train_indices = y_train[y_train != 0].index
print("non_zeros_train_indices", len(non_zeros_train_indices))

train_indices = np.append(np.array(zeros_keep_indices_train), np.array(non_zeros_train_indices))

y_train = y_train.loc[train_indices]
x_train = x_train.loc[train_indices]




x_val = training[training['date_block_num'] == 33]
y_val = x_val['y']

pos_val_len = len(y_val[y_val != 0])
print("pos_val_len", pos_val_len)

zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/6)).index
print("zeros_keep_indices_val", len(zeros_keep_indices_val))
non_zeros_val_indices = y_val[y_val != 0].index
print("non_zeros_val_indices", len(non_zeros_val_indices))

val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))

y_val = y_val.loc[val_indices]
x_val = x_val.loc[val_indices]

pos_train_len 444760
zeros_keep_indices_train 74126
non_zeros_train_indices 444760
pos_val_len 31471
zeros_keep_indices_val 5245
non_zeros_val_indices 31471


In [180]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'y', 'item_block_units',
       'level_1_x', 'item_block_units_rolling3', 'shop_block_units',
       'level_1_y', 'shop_block_units_rolling3', 'item_category_id',
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
       'shop_item_me', 'val_ignore'], dtype=object)

In [217]:



cb_features = ['item_me', 'shop_me',
      'category_me','shop_category_me', 'shop_item_me', 
'item_block_units_rolling3','shop_block_units_rolling3', 'rolling_composite', 'me_composite']

In [218]:
cb_model = CatBoostRegressor(iterations=70000,
                             learning_rate=0.01,
                             eval_metric='RMSE',
                             #thread_count=16,
                             task_type = "GPU",
                             use_best_model=True,
                             #l2_leaf_reg = 1000,
                             od_type = "Iter",
                             od_wait = 30,
                             #random_strength = 10,
                             #bagging_temperature = 1,
                             #one_hot_max_size = 2,
                             random_seed = 42)

#drops = ['subcategory','area']
#x_train = x_train.drop(columns=drops)
#x_val = x_val.drop(columns=drops)


cb_model.fit(x_train[cb_features], y_train, #cat_features=categorical_features_indices,
             eval_set=(x_val[cb_features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

0:	learn: 2.9634216	test: 2.9533624	best: 2.9533624 (0)	total: 5.64ms	remaining: 6m 35s
1:	learn: 2.9446387	test: 2.9390294	best: 2.9390294 (1)	total: 10.9ms	remaining: 6m 21s
2:	learn: 2.9260797	test: 2.9249410	best: 2.9249410 (2)	total: 16.2ms	remaining: 6m 18s
3:	learn: 2.9077963	test: 2.9109629	best: 2.9109629 (3)	total: 21.5ms	remaining: 6m 16s
4:	learn: 2.8897467	test: 2.8972663	best: 2.8972663 (4)	total: 26.6ms	remaining: 6m 13s
5:	learn: 2.8719205	test: 2.8839541	best: 2.8839541 (5)	total: 31.9ms	remaining: 6m 11s
6:	learn: 2.8543405	test: 2.8703319	best: 2.8703319 (6)	total: 38.5ms	remaining: 6m 24s
7:	learn: 2.8369708	test: 2.8569849	best: 2.8569849 (7)	total: 44.3ms	remaining: 6m 27s
8:	learn: 2.8198424	test: 2.8441309	best: 2.8441309 (8)	total: 49.8ms	remaining: 6m 27s
9:	learn: 2.8029252	test: 2.8312365	best: 2.8312365 (9)	total: 55.3ms	remaining: 6m 26s
10:	learn: 2.7862271	test: 2.8189276	best: 2.8189276 (10)	total: 61ms	remaining: 6m 28s
11:	learn: 2.7697200	test: 2.806

110:	learn: 1.8950352	test: 2.1146024	best: 2.1146024 (110)	total: 597ms	remaining: 6m 15s
111:	learn: 1.8911494	test: 2.1112379	best: 2.1112379 (111)	total: 602ms	remaining: 6m 15s
112:	learn: 1.8874319	test: 2.1077184	best: 2.1077184 (112)	total: 607ms	remaining: 6m 15s
113:	learn: 1.8837098	test: 2.1044038	best: 2.1044038 (113)	total: 611ms	remaining: 6m 14s
114:	learn: 1.8800558	test: 2.1008000	best: 2.1008000 (114)	total: 617ms	remaining: 6m 14s
115:	learn: 1.8764811	test: 2.0973473	best: 2.0973473 (115)	total: 623ms	remaining: 6m 15s
116:	learn: 1.8729649	test: 2.0943730	best: 2.0943730 (116)	total: 629ms	remaining: 6m 15s
117:	learn: 1.8694361	test: 2.0909367	best: 2.0909367 (117)	total: 634ms	remaining: 6m 15s
118:	learn: 1.8659613	test: 2.0877534	best: 2.0877534 (118)	total: 639ms	remaining: 6m 15s
119:	learn: 1.8625818	test: 2.0847031	best: 2.0847031 (119)	total: 644ms	remaining: 6m 14s
120:	learn: 1.8592335	test: 2.0818657	best: 2.0818657 (120)	total: 651ms	remaining: 6m 15s

210:	learn: 1.6959100	test: 1.9200506	best: 1.9200506 (210)	total: 1.2s	remaining: 6m 36s
211:	learn: 1.6949194	test: 1.9188104	best: 1.9188104 (211)	total: 1.21s	remaining: 6m 36s
212:	learn: 1.6940820	test: 1.9178439	best: 1.9178439 (212)	total: 1.21s	remaining: 6m 37s
213:	learn: 1.6930902	test: 1.9169164	best: 1.9169164 (213)	total: 1.22s	remaining: 6m 38s
214:	learn: 1.6922265	test: 1.9159551	best: 1.9159551 (214)	total: 1.23s	remaining: 6m 39s
215:	learn: 1.6913549	test: 1.9145955	best: 1.9145955 (215)	total: 1.24s	remaining: 6m 40s
216:	learn: 1.6905378	test: 1.9135953	best: 1.9135953 (216)	total: 1.25s	remaining: 6m 40s
217:	learn: 1.6897584	test: 1.9127107	best: 1.9127107 (217)	total: 1.25s	remaining: 6m 41s
218:	learn: 1.6888993	test: 1.9115814	best: 1.9115814 (218)	total: 1.26s	remaining: 6m 41s
219:	learn: 1.6880846	test: 1.9108993	best: 1.9108993 (219)	total: 1.27s	remaining: 6m 42s
220:	learn: 1.6872079	test: 1.9097951	best: 1.9097951 (220)	total: 1.28s	remaining: 6m 42s


336:	learn: 1.6336440	test: 1.8399602	best: 1.8399602 (336)	total: 2s	remaining: 6m 53s
337:	learn: 1.6333237	test: 1.8396634	best: 1.8396634 (337)	total: 2s	remaining: 6m 52s
338:	learn: 1.6330829	test: 1.8394505	best: 1.8394505 (338)	total: 2.01s	remaining: 6m 52s
339:	learn: 1.6328061	test: 1.8391599	best: 1.8391599 (339)	total: 2.01s	remaining: 6m 52s
340:	learn: 1.6325664	test: 1.8388663	best: 1.8388663 (340)	total: 2.02s	remaining: 6m 52s
341:	learn: 1.6322893	test: 1.8386405	best: 1.8386405 (341)	total: 2.02s	remaining: 6m 52s
342:	learn: 1.6320290	test: 1.8383163	best: 1.8383163 (342)	total: 2.03s	remaining: 6m 52s
343:	learn: 1.6317535	test: 1.8379721	best: 1.8379721 (343)	total: 2.03s	remaining: 6m 51s
344:	learn: 1.6314998	test: 1.8375757	best: 1.8375757 (344)	total: 2.04s	remaining: 6m 51s
345:	learn: 1.6312669	test: 1.8372050	best: 1.8372050 (345)	total: 2.04s	remaining: 6m 51s
346:	learn: 1.6309695	test: 1.8369588	best: 1.8369588 (346)	total: 2.05s	remaining: 6m 51s
347:	

434:	learn: 1.6135683	test: 1.8154831	best: 1.8154831 (434)	total: 2.6s	remaining: 6m 55s
435:	learn: 1.6134281	test: 1.8153880	best: 1.8153880 (435)	total: 2.6s	remaining: 6m 55s
436:	learn: 1.6132379	test: 1.8152020	best: 1.8152020 (436)	total: 2.61s	remaining: 6m 56s
437:	learn: 1.6130839	test: 1.8151035	best: 1.8151035 (437)	total: 2.62s	remaining: 6m 56s
438:	learn: 1.6129057	test: 1.8149716	best: 1.8149716 (438)	total: 2.63s	remaining: 6m 56s
439:	learn: 1.6126879	test: 1.8144235	best: 1.8144235 (439)	total: 2.64s	remaining: 6m 56s
440:	learn: 1.6125263	test: 1.8143083	best: 1.8143083 (440)	total: 2.65s	remaining: 6m 57s
441:	learn: 1.6123981	test: 1.8141375	best: 1.8141375 (441)	total: 2.65s	remaining: 6m 57s
442:	learn: 1.6122037	test: 1.8138709	best: 1.8138709 (442)	total: 2.66s	remaining: 6m 57s
443:	learn: 1.6120926	test: 1.8137883	best: 1.8137883 (443)	total: 2.67s	remaining: 6m 58s
444:	learn: 1.6118537	test: 1.8132954	best: 1.8132954 (444)	total: 2.68s	remaining: 6m 58s
4

...

4373:	learn: 1.4835813	test: 1.6651880	best: 1.6651854 (4372)	total: 22.8s	remaining: 5m 41s
4374:	learn: 1.4835598	test: 1.6651797	best: 1.6651797 (4374)	total: 22.8s	remaining: 5m 41s
4375:	learn: 1.4835395	test: 1.6651272	best: 1.6651272 (4375)	total: 22.8s	remaining: 5m 41s
4376:	learn: 1.4835321	test: 1.6651272	best: 1.6651272 (4375)	total: 22.8s	remaining: 5m 41s
4377:	learn: 1.4835143	test: 1.6651132	best: 1.6651132 (4377)	total: 22.8s	remaining: 5m 41s
4378:	learn: 1.4835014	test: 1.6651091	best: 1.6651091 (4378)	total: 22.8s	remaining: 5m 41s
4379:	learn: 1.4834921	test: 1.6651017	best: 1.6651017 (4379)	total: 22.8s	remaining: 5m 41s
4380:	learn: 1.4834841	test: 1.6651032	best: 1.6651017 (4379)	total: 22.8s	remaining: 5m 41s
4381:	learn: 1.4834666	test: 1.6650927	best: 1.6650927 (4381)	total: 22.8s	remaining: 5m 41s
4382:	learn: 1.4834316	test: 1.6649846	best: 1.6649846 (4382)	total: 22.8s	remaining: 5m 41s
4383:	learn: 1.4834108	test: 1.6649755	best: 1.6649755 (4383)	total: 2

4492:	learn: 1.4820743	test: 1.6643309	best: 1.6643309 (4492)	total: 23.4s	remaining: 5m 40s
4493:	learn: 1.4820587	test: 1.6643353	best: 1.6643309 (4492)	total: 23.4s	remaining: 5m 40s
4494:	learn: 1.4820535	test: 1.6643327	best: 1.6643309 (4492)	total: 23.4s	remaining: 5m 40s
4495:	learn: 1.4820418	test: 1.6643267	best: 1.6643267 (4495)	total: 23.4s	remaining: 5m 40s
4496:	learn: 1.4820376	test: 1.6643124	best: 1.6643124 (4496)	total: 23.4s	remaining: 5m 40s
4497:	learn: 1.4820273	test: 1.6643095	best: 1.6643095 (4497)	total: 23.4s	remaining: 5m 40s
4498:	learn: 1.4820166	test: 1.6643112	best: 1.6643095 (4497)	total: 23.4s	remaining: 5m 40s
4499:	learn: 1.4820083	test: 1.6643125	best: 1.6643095 (4497)	total: 23.4s	remaining: 5m 40s
4500:	learn: 1.4819876	test: 1.6642924	best: 1.6642924 (4500)	total: 23.4s	remaining: 5m 40s
4501:	learn: 1.4819668	test: 1.6643074	best: 1.6642924 (4500)	total: 23.4s	remaining: 5m 40s
4502:	learn: 1.4819532	test: 1.6642937	best: 1.6642924 (4500)	total: 2

4610:	learn: 1.4805911	test: 1.6637895	best: 1.6637872 (4608)	total: 23.9s	remaining: 5m 39s
4611:	learn: 1.4805792	test: 1.6637860	best: 1.6637860 (4611)	total: 23.9s	remaining: 5m 39s
4612:	learn: 1.4805707	test: 1.6637833	best: 1.6637833 (4612)	total: 23.9s	remaining: 5m 39s
4613:	learn: 1.4805485	test: 1.6637406	best: 1.6637406 (4613)	total: 24s	remaining: 5m 39s
4614:	learn: 1.4805428	test: 1.6637407	best: 1.6637406 (4613)	total: 24s	remaining: 5m 39s
4615:	learn: 1.4805317	test: 1.6637448	best: 1.6637406 (4613)	total: 24s	remaining: 5m 39s
4616:	learn: 1.4805267	test: 1.6637407	best: 1.6637406 (4613)	total: 24s	remaining: 5m 39s
4617:	learn: 1.4805170	test: 1.6637340	best: 1.6637340 (4617)	total: 24s	remaining: 5m 39s
4618:	learn: 1.4804949	test: 1.6637124	best: 1.6637124 (4618)	total: 24s	remaining: 5m 39s
4619:	learn: 1.4804827	test: 1.6637074	best: 1.6637074 (4619)	total: 24s	remaining: 5m 39s
4620:	learn: 1.4804725	test: 1.6636933	best: 1.6636933 (4620)	total: 24s	remaining: 

4729:	learn: 1.4792318	test: 1.6631154	best: 1.6631154 (4729)	total: 24.5s	remaining: 5m 38s
4730:	learn: 1.4792211	test: 1.6631030	best: 1.6631030 (4730)	total: 24.5s	remaining: 5m 38s
4731:	learn: 1.4792063	test: 1.6631138	best: 1.6631030 (4730)	total: 24.5s	remaining: 5m 38s
4732:	learn: 1.4791954	test: 1.6631123	best: 1.6631030 (4730)	total: 24.5s	remaining: 5m 38s
4733:	learn: 1.4791892	test: 1.6631095	best: 1.6631030 (4730)	total: 24.5s	remaining: 5m 38s
4734:	learn: 1.4791658	test: 1.6630533	best: 1.6630533 (4734)	total: 24.6s	remaining: 5m 38s
4735:	learn: 1.4791564	test: 1.6630533	best: 1.6630533 (4734)	total: 24.6s	remaining: 5m 38s
4736:	learn: 1.4791440	test: 1.6630633	best: 1.6630533 (4734)	total: 24.6s	remaining: 5m 38s
4737:	learn: 1.4791316	test: 1.6630575	best: 1.6630533 (4734)	total: 24.6s	remaining: 5m 38s
4738:	learn: 1.4791116	test: 1.6630431	best: 1.6630431 (4738)	total: 24.6s	remaining: 5m 38s
4739:	learn: 1.4791057	test: 1.6630377	best: 1.6630377 (4739)	total: 2

4825:	learn: 1.4780224	test: 1.6621285	best: 1.6621213 (4824)	total: 25.1s	remaining: 5m 39s
4826:	learn: 1.4780104	test: 1.6621294	best: 1.6621213 (4824)	total: 25.1s	remaining: 5m 39s
4827:	learn: 1.4779959	test: 1.6620969	best: 1.6620969 (4827)	total: 25.1s	remaining: 5m 39s
4828:	learn: 1.4779874	test: 1.6620795	best: 1.6620795 (4828)	total: 25.1s	remaining: 5m 39s
4829:	learn: 1.4779575	test: 1.6620139	best: 1.6620139 (4829)	total: 25.1s	remaining: 5m 39s
4830:	learn: 1.4779415	test: 1.6620023	best: 1.6620023 (4830)	total: 25.1s	remaining: 5m 39s
4831:	learn: 1.4779328	test: 1.6620050	best: 1.6620023 (4830)	total: 25.1s	remaining: 5m 39s
4832:	learn: 1.4779270	test: 1.6620044	best: 1.6620023 (4830)	total: 25.1s	remaining: 5m 39s
4833:	learn: 1.4779189	test: 1.6620002	best: 1.6620002 (4833)	total: 25.2s	remaining: 5m 39s
4834:	learn: 1.4779119	test: 1.6619967	best: 1.6619967 (4834)	total: 25.2s	remaining: 5m 39s
4835:	learn: 1.4778944	test: 1.6620018	best: 1.6619967 (4834)	total: 2

In [148]:
#training.to_pickle("pickled/training")
#training = pd.read_pickle("pickled/training")

#pickle.dump(cb_model, open( "pickled/cb_model", "wb"), protocol=4)

#cb_model = pickle.load( open( "pickled/cb_model", "rb" ) )

In [219]:
scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    scores[cb_features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

[('rolling_composite', 36.13121603410879),
 ('shop_item_me', 21.575255421557912),
 ('item_block_units_rolling3', 8.786720526790265),
 ('shop_category_me', 8.289838595704609),
 ('me_composite', 6.617920257920725),
 ('category_me', 5.231669789811722),
 ('item_me', 4.7092542054265305),
 ('shop_block_units_rolling3', 4.468709879955291),
 ('shop_me', 4.189415288724149)]

[('rolling_composite', 36.09854669529227),
 ('shop_item_me', 22.429519800926272),
 ('item_me', 9.671672250370097),
 ('shop_category_me', 8.74728541130911),
 ('item_block_units_rolling3', 7.944012192102696),
 ('shop_block_units_rolling3', 5.822284914719897),
 ('category_me', 5.1903530084861655),
 ('shop_me', 4.096325726793502)]

In [203]:
cb_features = [item[0] for item in scores.items() if item[1] > 9]